Estimate deltaO18_sw from the data

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pymc as pm

In [ ]:
from lgmproxies.datasets.tierney import get_repo_path
repo = get_repo_path("brews/d18oc_sst")

In [ ]:
coretops_raw = pd.read_csv(repo/"data/parsed/coretops.csv")
coretops_grid = pd.read_csv(repo/"data/parsed/coretops_grid.csv")

In [ ]:
coretops_raw['foramtype'] = coretops_raw['species'].astype('category')
coretops_grid['foramtype'] = coretops_grid['species'].astype('category')

In [ ]:
from lgmproxies.datasets.naturalearth import add_coast

fig, axes = plt.subplots(2, 1, figsize=(10, 10))

axes[0].scatter(coretops_raw.gridlon, coretops_raw.gridlat,c=coretops_raw.foramtype.cat.codes,s=1)
add_coast(ax=axes[0])
axes[0].set_title("Coretops Raw Data")
axes[1].scatter(coretops_grid.gridlon, coretops_grid.gridlat,c=coretops_grid.foramtype.cat.codes,s=1)
add_coast(ax=axes[1])
axes[1].set_title("Coretops Grid Data")

In [ ]:
coretops_raw

In [ ]:
coretops_grid

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
ax = axes[0]
ax.hist(coretops_raw.gridlon, label='Raw', alpha=0.5, bins=50)
ax.hist(coretops_grid.gridlon, label='Grid', alpha=0.5, bins=50)
# ax.scatter(coretops_raw.gridlon, coretops_grid.gridlon)
ax.set_xlabel("Longitude")
ax.set_ylabel("Count")
ax.legend()
ax = axes[1]
ax.hist(coretops_raw.gridlat, label='Raw', alpha=0.5, bins=50)
ax.hist(coretops_grid.gridlat, label='Grid', alpha=0.5, bins=50)
# ax.scatter(coretops_raw.gridlat, coretops_grid.gridlat)
ax.set_xlabel("Latitude")
ax.set_ylabel("Count")
ax.legend()
fig.tight_layout()

In [ ]:
from lgmproxies.datasets.naturalearth import add_coast

fig, ax = plt.subplots(1, 1, figsize=(10, 5))

df = coretops_raw
h = ax.scatter(df.gridlon, df.gridlat,
    c=df.d18osw,s=1)
add_coast(ax=ax)
ax.set_title("Coretops Raw Data")
plt.colorbar(h, ax=ax, label='d18O seawater (per mil)')

In [ ]:
from lgmproxies.gaskell_hull2023 import convert_d18o_df

input_df = coretops_raw[["gridlat", "gridlon", "d18oc"]].rename(
    columns={
        "gridlat": "lat",
        "gridlon": "long",
        "d18oc": "d18O"
    }
)
input_df["age"] = 0 * 1e-6  # Convert to Ma
chunk_size = len(input_df) // 5
chunks = [input_df.iloc[i:i+chunk_size] for i in range(0, len(input_df), chunk_size)]
coretops_gh23 = pd.concat(
    [convert_d18o_df(chunk) for chunk in chunks],
)
coretops_gh23

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

ax = axes[0]
# h = ax.scatter(coretops_raw.d18osw.values, coretops_gh23.d18Osw_spatial.values, s=1)
h = ax.scatter(coretops_raw.d18osw.values, (
    coretops_gh23.d18Osw_spatial.values +
    coretops_gh23.d18Osw_global.values
    ), s=1)
ax.set_aspect('equal')
xlim = ax.get_xlim()
ax.plot(xlim, xlim, 'k--', label='y=x')
ax.set_title("d18O seawater (per mil)")
ax.set_xlabel("brews/d18oc_sst")
ax.set_ylabel("Gaskell and Hull 2023")


ax = axes[1]
h = ax.scatter(coretops_raw.t_annual.values, (
    coretops_gh23.temp.values
    ), s=1)
ax.set_aspect('equal')
xlim = ax.get_xlim()
ax.plot(xlim, xlim, 'k--', label='y=x')
ax.set_title("Annual SST (C)")
ax.set_xlabel("brews/d18oc_sst")
ax.set_ylabel("Gaskell and Hull 2023")
fig.tight_layout()
fig.savefig("../images/d18osw_comparison_scatter.png", dpi=300)

In [ ]:
from lgmproxies.datasets.naturalearth import add_coast

fig, axes = plt.subplots(2, 1, figsize=(10, 10))

df = coretops_raw

ax = axes[1]
h = ax.scatter(df.gridlon, df.gridlat,
    c=coretops_gh23.d18Osw_spatial.values,s=1,
    vmin=coretops_raw.d18osw.min(), vmax=coretops_raw.d18osw.max())
add_coast(ax=ax)
ax.set_title("brews/d18oc_sst (d18o+lat+age=0) -> Gaskell and Hull 2023")
plt.colorbar(h, ax=ax, label='d18O seawater (per mil)')

ax = axes[0]
h = ax.scatter(df.gridlon, df.gridlat,
    c=df.d18osw,s=1,
    vmin=coretops_raw.d18osw.min(), vmax=coretops_raw.d18osw.max())
add_coast(ax=ax)
ax.set_title("Coretops Raw Data (brews/d18oc_sst)")
plt.colorbar(h, ax=ax, label='d18O seawater (per mil)')

fig.tight_layout()
fig.savefig("../images/d18osw_comparison.png", dpi=300)